In [1]:
import mxnet as mx
from mxnet import nd, autograd
import numpy as np
mx.random.seed(2)
ctx = mx.cpu()

with open("nlg/mifid2.txt") as f:
    mifid = f.read()

C:\Users\john\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\Users\john\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\contrib\pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
character_list = list(set(mifid))
vocab_size = len(character_list)
character_dict = {}
for e, char in enumerate(character_list):
    character_dict[char] = e
time_numerical = [character_dict[char] for char in mifid]

In [3]:
def one_hots(numerical_list, vocab_size=vocab_size):
    result = nd.zeros((len(numerical_list), vocab_size), ctx=ctx)
    for i, idx in enumerate(numerical_list):
        result[i, idx] = 1.0
    return result
def textify(embedding):
    result = ""
    indices = nd.argmax(embedding, axis=1).asnumpy()
    for idx in indices:
        result += character_list[int(idx)]
    return result

In [4]:
batch_size = 32
seq_length = 64
num_samples = (len(time_numerical) - 1) // seq_length
dataset = one_hots(time_numerical[:seq_length*num_samples]).reshape((num_samples, seq_length, vocab_size))
num_batches = len(dataset) // batch_size
train_data = dataset[:num_batches*batch_size].reshape((num_batches, batch_size, seq_length, vocab_size))
# swap batch_size and seq_length axis to make later access easier
train_data = nd.swapaxes(train_data, 1, 2)

labels = one_hots(time_numerical[1:seq_length*num_samples+1])
train_label = labels.reshape((num_batches, batch_size, seq_length, vocab_size))
train_label = nd.swapaxes(train_label, 1, 2)

In [5]:
num_inputs = vocab_size
num_hidden = 256
num_outputs = vocab_size

########################
#  Weights connecting the inputs to the hidden layer
########################
Wxg = nd.random_normal(shape=(num_inputs,num_hidden), ctx=ctx) * .01
Wxi = nd.random_normal(shape=(num_inputs,num_hidden), ctx=ctx) * .01
Wxf = nd.random_normal(shape=(num_inputs,num_hidden), ctx=ctx) * .01
Wxo = nd.random_normal(shape=(num_inputs,num_hidden), ctx=ctx) * .01

########################
#  Recurrent weights connecting the hidden layer across time steps
########################
Whg = nd.random_normal(shape=(num_hidden,num_hidden), ctx=ctx)* .01
Whi = nd.random_normal(shape=(num_hidden,num_hidden), ctx=ctx)* .01
Whf = nd.random_normal(shape=(num_hidden,num_hidden), ctx=ctx)* .01
Who = nd.random_normal(shape=(num_hidden,num_hidden), ctx=ctx)* .01

########################
#  Bias vector for hidden layer
########################
bg = nd.random_normal(shape=num_hidden, ctx=ctx) * .01
bi = nd.random_normal(shape=num_hidden, ctx=ctx) * .01
bf = nd.random_normal(shape=num_hidden, ctx=ctx) * .01
bo = nd.random_normal(shape=num_hidden, ctx=ctx) * .01

########################
# Weights to the output nodes
########################
Why = nd.random_normal(shape=(num_hidden,num_outputs), ctx=ctx) * .01
by = nd.random_normal(shape=num_outputs, ctx=ctx) * .01

In [6]:
params = [Wxg, Wxi, Wxf, Wxo, Whg, Whi, Whf, Who, bg, bi, bf, bo, Why, by]

for param in params:
    param.attach_grad()
def softmax(y_linear, temperature=1.0):
    lin = (y_linear-nd.max(y_linear)) / temperature
    exp = nd.exp(lin)
    partition = nd.sum(exp, axis=0, exclude=True).reshape((-1,1))
    return exp / partition
def lstm_rnn(inputs, h, c, temperature=1.0):
    outputs = []
    for X in inputs:
        g = nd.tanh(nd.dot(X, Wxg) + nd.dot(h, Whg) + bg)
        i = nd.sigmoid(nd.dot(X, Wxi) + nd.dot(h, Whi) + bi)
        f = nd.sigmoid(nd.dot(X, Wxf) + nd.dot(h, Whf) + bf)
        o = nd.sigmoid(nd.dot(X, Wxo) + nd.dot(h, Who) + bo)
        #######################
        #
        #######################
        c = f * c + i * g
        h = o * nd.tanh(c)
        #######################
        #
        #######################
        yhat_linear = nd.dot(h, Why) + by
        yhat = softmax(yhat_linear, temperature=temperature)
        outputs.append(yhat)
    return (outputs, h, c)

In [7]:
def cross_entropy(yhat, y):
    return - nd.mean(nd.sum(y * nd.log(yhat), axis=0, exclude=True))

def average_ce_loss(outputs, labels):
    assert(len(outputs) == len(labels))
    total_loss = 0.
    for (output, label) in zip(outputs,labels):
        total_loss = total_loss + cross_entropy(output, label)
    return total_loss / len(outputs)
def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad
        
def sample(prefix, num_chars, temperature=1.0):
    #####################################
    # Initialize the string that we'll return to the supplied prefix
    #####################################
    string = prefix

    #####################################
    # Prepare the prefix as a sequence of one-hots for ingestion by RNN
    #####################################
    prefix_numerical = [character_dict[char] for char in prefix]
    input = one_hots(prefix_numerical)

    #####################################
    # Set the initial state of the hidden representation ($h_0$) to the zero vector
    #####################################
    h = nd.zeros(shape=(1, num_hidden), ctx=ctx)
    c = nd.zeros(shape=(1, num_hidden), ctx=ctx)

    #####################################
    # For num_chars iterations,
    #     1) feed in the current input
    #     2) sample next character from from output distribution
    #     3) add sampled character to the decoded string
    #     4) prepare the sampled character as a one_hot (to be the next input)
    #####################################
    for i in range(num_chars):
        outputs, h, c = lstm_rnn(input, h, c, temperature=temperature)
        choice = np.random.choice(vocab_size, p=outputs[-1][0].asnumpy())
        string += character_list[choice]
        input = one_hots([choice])
    return string

In [8]:
epochs = 50
moving_loss = 0.

learning_rate = 2.0

# state = nd.zeros(shape=(batch_size, num_hidden), ctx=ctx)
for e in range(epochs):
    ############################
    # Attenuate the learning rate by a factor of 2 every 10 epochs.
    ############################
    if ((e+1) % 10 == 0):
        learning_rate = learning_rate / 2.0
    h = nd.zeros(shape=(batch_size, num_hidden), ctx=ctx)
    c = nd.zeros(shape=(batch_size, num_hidden), ctx=ctx)
    for i in range(num_batches):
        data_one_hot = train_data[i]
        label_one_hot = train_label[i]
        with autograd.record():
            outputs, h, c = lstm_rnn(data_one_hot, h, c)
            loss = average_ce_loss(outputs, label_one_hot)
            loss.backward()
        SGD(params, learning_rate)

        ##########################
        #  Keep a moving average of the losses
        ##########################
        if (i == 0) and (e == 0):
            moving_loss = nd.mean(loss).asscalar()
        else:
            moving_loss = .99 * moving_loss + .01 * nd.mean(loss).asscalar()

    print("Epoch %s. Loss: %s" % (e, moving_loss))
    print(sample("The committee", 1024,temperature=.1 ))


Epoch 0. Loss: 3.2066613358610434
The committee te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te on te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te tin on te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te tin te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te tite te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te on te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te te 

Epoch 8. Loss: 1.3270543673914932
The committee and the contract of the consures or the consult the contract of the consitions of the consitions of the provessional compers of the consures and the contract of the contract of the contract of the contract of the provessional compers to the propessional consures and the contract of the provessional compers of the contract of the contract of the contract of the contract of the contract of the prope of the consures or the contract and in the consider the considers to the contract of the consitions of the contract of the contract of the contract of the contract of the provessional compers to the provessional contract to the contract of the consitions of the considers to the contract of the contract of the consures or the consures or the provessional compers to the consitions of the consitions of the consures or the consitions of the contract and of the contract of the provessional investment firm of the contract and the contract of the contr

Epoch 16. Loss: 1.0592461166681233
The committee investment firms and provided to the condetions of the competent authority of the competent authorities of the conditions of the conditions of the contract in the first subparagraph in accordance with pritective and the contract subparagraph in accordance with the conditions of the conditions of the contract subparagraph in accordance with the conditions in the provisions of the competent authorities of the conditions of the conditions of the conditions of the conditions of the conditions of the conditions of the competent authorities of the conditions of the conditions of the contract subparagraph in accordance with the conditions of the conditions of the competent authorities of the conditions in the first subparagraph in accordance with prites and the contract subparagraph in accordance with the conditions in the contract subparagraph in accordance with the conditions of the conditions of the conditions of the conditions of the condit

Epoch 24. Loss: 0.9464266509444252
The committee investment firms and provided for the provisions of the competent authorities and the conditions for the provisions of the conditions of the conditions for the provisions of the conditions of the conditions of the conditions of the conditions for the purpose of provided to the competent authority of the competent authorities and the conditions for the provisions of the conditions in the content of the competent authorities of the competent authority of the competent authority of the competent authority of the competent authority of the competent authorities of the conditions for the provisions of the conditions for the provisions of the competent authority of the competent authority of the competent authority of the competent authorities of the competent authority of the competent authority of the competent authority of the competent authorities of the conditions for the purposes of the conditions for the purpose of provided for the prov

Epoch 32. Loss: 0.8881984359327498
The committee the contract subparagraph of Article 1(1) of Regulation (EU) No 600/2014 of the European Parliament and of the Council of the subparagraph of Article 21(1) and (E) No 100/2010 of the European Parliament and of the Council of the competent authority of the home Member State of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the competent authority of the home Member States the

Epoch 40. Loss: 0.8553342217407224
The committee the provisions of the provisions of the provisions of the provisions of the provisions of the provision of investment firms and market operators of the competent authority of the home Member State of the competent authority of the home Member State or the provisions of the provisions of the provisions of the provisions of the provisions of the provisions of the provisions of the provisions of the provisions of the provisions of the provisions of the provision of investment firms and provided to the competent authority of the home Member State or the provisions of the provisions of the provision of investment firms and market operators operating an MTF or an OTF shall be empowered to adopt delegated acts in accordance with Article 19 of Regulation (EU) No 600/2014 of the European Parliament and of the Council of 14 July 2014 on the transactions executed or referred to in the first subparagraph in accordance with Article 14 of Regulation (

Epoch 48. Loss: 0.8388374733011212
The committee the investment firm and the provisions of the provisions of the provisions of the provisions of the provisions of the protection of the competent authorities of the competent authority of the home Member State or the provisions of the protection of the competent authorities of the competent authority of the home Member State or the provisions of the provisions of the provision of investment firms and market operators operating an MTF or an OTF shall be empowered to adopt delegated acts in accordance with Article 19 of Regulation (EU) No 600/2014 of the European Parliament and of the Council of 2 July 2015 concluded on the provisions of the provisions of the provisions of the protection of the competent authority of the home Member State or the provisions of the provisions of the protection of the conditions for information to be able to considered to be able to ensure that the competent authority of the home Member State or the provision